## Set up TensorFlow

Import TensorFlow into your program to get started:

In [ ]:
import numpy
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dense, Reshape, Flatten
from keras import initializers
from keras import regularizers
from keras import constraints
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.8.2


In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


Activations

In machine learning, activation function is a special function used to find whether a specific neuron is activated or not. Basically, the activation function does a nonlinear transformation of the input data and thus enable the neurons to learn better. Output of a neuron depends on the activation function.


In [ ]:
#creates a new model using Sequential API.
model = Sequential()
# creates a new Dense layer and add it into the model.
# Dense is an entry level layer provided by Keras,
# which accepts the number of neurons or units (32) as its required parameter.
# If the layer is first layer, then we need to provide Input Shape, (16,) as well.
# Otherwise, the output of the previous layer will be used as input of the next layer.

# input_shape represent the shape of input data.
# kernel_initializer represent initializer to be used.
#    he_uniform function is set as value.
# kernel_regularizer represent regularizer to be used.
#    None is set as value.
# kernel_constraint represent constraint to be used.
#    MaxNorm function is set as value.
# activation represent activation to be used.
# relu function is set as value.

model.add(Dense(32, input_shape=(16,), kernel_initializer = 'he_uniform',
   kernel_regularizer = None, kernel_constraint = 'MaxNorm', activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
#creates final Dense layer with 8 units.
model.add(Dense(8))

In [ ]:
#Generates 0 for all inputs

my_init = initializers.Zeros()
model = Sequential()
model.add(Dense(512, activation = 'relu', input_shape = (784,),
   kernel_initializer = my_init))


In [ ]:
#Generates 1 for all inputs

my_init = initializers.Ones()
model.add(Dense(512, activation = 'relu', input_shape = (784,),
   kernel_initializer = my_init))

In [ ]:
#Generate value using uniform distribution of input data

my_init = initializers.RandomUniform(minval = -0.05, maxval = 0.05, seed = None)
model.add(Dense(512, activation = 'relu', input_shape = (784,),
   kernel_initializer = my_init))

# minval represent the lower bound of the random values to generate
# maxval represent the upper bound of the random values to generate

In [ ]:
#Generates value based on the input shape and output shape of the layer along with the specified scale.

my_init = initializers.VarianceScaling(
   scale = 1.0, mode = 'fan_in', distribution = 'normal', seed = None)
model.add(Dense(512, activation = 'relu', input_shape = (784,),
   kernel_initializer = my_init))

# fan_in represents the number of input units
# fan_out represents the number of output units
# scale represent the scaling factor
# mode represent any one of fan_in, fan_out and fan_avg values
# distribution represent either of normal or uniform

In [ ]:
#Generates identity matrix

my_init = initializers.Identity(gain = 1.0)
model.add(Dense(512, activation = 'relu', input_shape = (784,), kernel_initializer = my_init))

In [ ]:
#Constraints weights to be unit form

my_constrain = constraints.UnitNorm(axis = 0)
model = Sequential()
model.add(Dense(512, activation = 'relu', input_shape = (784,),
   kernel_constraint = my_constrain))

In [ ]:
#Constraints weight to norm less than or equals to the given value.

my_constrain = constraints.MaxNorm(max_value = 2, axis = 0)
model = Sequential()
model.add(Dense(512, activation = 'relu', input_shape = (784,),
   kernel_constraint = my_constrain))

In [ ]:
# Applying Sigmoid function

model = Sequential()
model.add(Dense(512, activation = 'sigmoid', input_shape = (784,)))

In [ ]:
#Applies Rectified Linear Unit.

model = Sequential()
model.add(Dense(512, activation = 'relu', input_shape = (784,)))

In [ ]:
#Applies Scaled exponential linear unit.

model = Sequential()
model.add(Dense(512, activation = 'selu', input_shape = (784,)))

In [ ]:
#Reshape

model = Sequential()
layer_1 = Dense(16, input_shape = (8,8))
model.add(layer_1)
layer_2 = Reshape((16, 8))
model.add(layer_2)


In [ ]:
#Flattern

model = Sequential()
layer_1 = Dense(16, input_shape=(8,8))
model.add(layer_1)
layer_2 = Flatten()
model.add(layer_2)

## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.10342179, -0.00837833, -0.8590822 ,  0.5979148 ,  0.29961196,
         0.9529247 , -0.26174614, -0.94969255, -0.47442734, -0.31852612]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.08519964, 0.09369461, 0.0400183 , 0.1718005 , 0.12748912,
        0.24502096, 0.0727241 , 0.03655166, 0.05879117, 0.06870987]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

1.4064115

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2909 - accuracy: 0.9156
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1373 - accuracy: 0.9600
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1027 - accuracy: 0.9688
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0854 - accuracy: 0.9739
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0717 - accuracy: 0.9772


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0745 - accuracy: 0.9780 - 577ms/epoch - 2ms/step


[0.07446999847888947, 0.9779999852180481]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.48997036e-08, 2.56449653e-08, 1.78168364e-06, 6.48483365e-06,
        7.70344032e-12, 1.88325675e-08, 3.64557213e-12, 9.99989986e-01,
        3.72552648e-07, 1.45681770e-06],
       [1.73849543e-07, 1.04438936e-04, 9.99894381e-01, 8.47589774e-07,
        8.70311216e-18, 1.19621063e-07, 1.14045498e-07, 3.07103378e-15,
        7.51335527e-09, 3.75649422e-14],
       [1.34470349e-06, 9.99006450e-01, 1.37658586e-04, 7.24233541e-06,
        4.10906177e-05, 3.13681085e-05, 6.11862561e-05, 6.57299359e-04,
        5.48781900e-05, 1.59907245e-06],
       [9.99809802e-01, 1.23450916e-10, 8.14087034e-05, 1.89384153e-08,
        3.10834594e-06, 1.05911988e-06, 9.30924944e-05, 1.13771575e-05,
        2.36483100e-08, 4.11857286e-08],
       [1.95905136e-06, 3.65665748e-10, 1.75986622e-06, 6.43656461e-09,
        9.98768628e-01, 2.66585722e-08, 2.18043042e-05, 1.32077432e-04,
        1.39968307e-07, 1.07354496e-03]], dtype=float32)>